In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data =  pd.read_csv('combo.gzip',compression='gzip',parse_dates=True,index_col='date')
print('done');

done


In [15]:
data.arrival_schedule = data.arrival_schedule.fillna(value=(data.arrival_actual + data.arrival_delay))
data.departure_schedule = data.departure_schedule.fillna(value=(data.departure_actual + data.departure_delay))

In [16]:
#departure&arrival_schedule are now not null and int32

In [17]:
for column in data:
    assert pd.notnull(data[column]).all()
    print(column+' is clean')

airline is clean
airline_code is clean
departure_airport is clean
departure_state is clean
departure_lat is clean
departure_lon is clean
arrival_airport is clean
arrival_state is clean
arrival_lat is clean
arrival_lon is clean
departure_schedule is clean
departure_actual is clean
departure_delay is clean
arrival_schedule is clean
arrival_actual is clean
arrival_delay is clean


In [18]:
data['airline_code'] = data['airline_code'].astype(str)
data['departure_delay'] = data['departure_delay'].astype(float)
data['arrival_delay'] = data['arrival_delay'].astype(float)

In [19]:
data = data.reset_index()

In [20]:
def formatTime(string):
    if len(string) >= 4:
        return string[0]+string[1]+':'+string[2]+string[3]
    elif len(string) == 3 :
        return '0'+string[0]+':'+string[1]+string[2]
    elif len(string) == 2:
        return '00:'+string[0]+string[1]
    elif len(string) == 1:
        return '00:0'+string[0]

In [21]:
print(formatTime('918'))

09:18


In [22]:
data['arrival_schedule'] = data['arrival_schedule'].astype(int)
data['departure_schedule'] = data['departure_schedule'].astype(int)
data['arrival_schedule'] = data['arrival_schedule'].astype(str)
data['departure_schedule'] = data['departure_schedule'].astype(str)
data['arrival_actual'] = data['arrival_actual'].astype(str)
data['departure_actual'] = data['departure_actual'].astype(str)

In [23]:
data.departure_schedule = data.departure_schedule.apply(formatTime)
data.departure_actual = data.departure_actual.apply(formatTime)
data.arrival_schedule = data.arrival_schedule.apply(formatTime)
data.arrival_actual = data.arrival_actual.apply(formatTime)

In [ ]:
data.head(20)

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,arrival_lon,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay
0,2012-11-25,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,17:55,18:05,10.0,19:31,19:48,17.0
1,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,17:55,17:52,-3.0,19:31,19:29,-2.0
2,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,14:45,14:42,-3.0,16:23,16:15,-8.0
3,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,20:05,20:07,2.0,21:40,21:39,-1.0
4,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,16:30,16:24,-6.0,18:08,18:05,-3.0
5,2012-11-30,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,22:20,22:13,-7.0,23:54,23:39,-15.0
6,2012-11-30,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,09:20,12:23,183.0,10:56,14:03,187.0
7,2012-11-26,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,20:05,20:06,1.0,21:40,21:44,4.0
8,2012-11-27,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,11:20,11:20,0.0,12:55,13:26,31.0
9,2012-11-27,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,13:10,13:09,-1.0,14:50,14:54,4.0


In [ ]:
data.to_csv('cleaned.gzip',compression='gzip')